In order to download only a subset of the COCO dataset we need to:

1- Download the whole dataset

2- Create a dataset that select only some categories and it copies the images into a folder "data"

3- Move the data folder to your HPC 

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz

# Download the COCO-2017 validation split and load it into FiftyOne
dataset = foz.load_zoo_dataset("coco-2017", split="validation")

# Give the dataset a new name, and make it persistent
dataset.name = "coco-2017-validation"
dataset.persistent = True


Found annotations at '/Users/germa/fiftyone/coco-2017/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'coco-2017-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [3]:
dataset_train = foz.load_zoo_dataset("coco-2017", split="train")
dataset_train.name = "coco-2017-train"
dataset_train.persistent = True

Found annotations at '/Users/germa/fiftyone/coco-2017/raw/instances_train2017.json'
Images already downloaded
Existing download of split 'train' is sufficient
Loading existing dataset 'coco-2017-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [4]:
dataset_test = foz.load_zoo_dataset("coco-2017", split="test")
dataset_test.name = "coco-2017-test"
dataset_test.persistent = True

Found test info at '/Users/germa/fiftyone/coco-2017/raw/image_info_test2017.json'
Images already downloaded
Existing download of split 'test' is sufficient
Loading existing dataset 'coco-2017-test'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [5]:
import json
import os
import numpy as np
import matplotlib.patches as patches
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from pycocotools.coco import COCO
from PIL import Image
import os
import numpy as np
import cv2  # OpenCV needed for polygon drawing
from pycocotools import mask as coco_mask
from matplotlib import pyplot as plt
import shutil

config = json.load(open("config.json"))

In [6]:
class CocoSegmentationDatasetMRCNN(Dataset):
    def __init__(self, image_dir, seg_annotation_file, categories_to_keep=[1], min_area_threshold=655, output_dir=None):
        self.image_dir = image_dir
        self.coco_seg = COCO(seg_annotation_file)
        self.min_area_threshold = min_area_threshold
        self.categories_to_keep = categories_to_keep
        self.output_dir = output_dir
        
        # Filter images to keep only those containing objects from specified categories
        self.image_ids = []
        for cat_id in self.categories_to_keep:
            ann_ids = self.coco_seg.getAnnIds(catIds=[cat_id], iscrowd=False)
            anns = self.coco_seg.loadAnns(ann_ids)
            valid_anns = [ann for ann in anns if ann['area'] >= self.min_area_threshold]
            img_ids = list(set([ann['image_id'] for ann in valid_anns]))
            self.image_ids.extend(img_ids)
        
        # Remove duplicates
        self.image_ids = list(set(self.image_ids))
        print(f"Dataset contains {len(self.image_ids)} images with categories {categories_to_keep}")
        
        # For visualization, create a category mapping
        self.category_map = {}
        for cat_id in self.categories_to_keep:
            cat_info = self.coco_seg.loadCats(cat_id)[0]
            self.category_map[cat_id] = cat_info['name']
        
        # Copy images to output directory if specified
        if output_dir:
            self.copy_images_to_output_dir()

    def copy_images_to_output_dir(self):
        """Copy filtered images to the output directory"""
        if not self.output_dir:
            return
            
        # Create output directory
        os.makedirs(self.output_dir, exist_ok=True)
        
        print(f"Copying {len(self.image_ids)} images to {self.output_dir}...")
        copied_count = 0
        
        for img_id in self.image_ids:
            # Get image info
            img_info = self.coco_seg.loadImgs(img_id)[0]
            
            # Source and destination paths
            src_path = os.path.join(self.image_dir, img_info["file_name"])
            dst_path = os.path.join(self.output_dir, img_info["file_name"])
            
            # Copy image if source exists
            if os.path.exists(src_path):
                os.makedirs(os.path.dirname(dst_path), exist_ok=True)
                shutil.copy(src_path, dst_path)
                copied_count += 1
            else:
                print(f"Warning: Could not find {src_path}")
        
        print(f"Copied {copied_count} images to {self.output_dir}")

    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        
        # Load image
        image_info = self.coco_seg.loadImgs(image_id)[0]
        image_path = os.path.join(self.image_dir, image_info["file_name"])
        image = Image.open(image_path).convert("RGB")
        
        # Convert to tensor
        image = transforms.ToTensor()(image)
        
        # Load annotations
        ann_ids = self.coco_seg.getAnnIds(imgIds=image_id, catIds=self.categories_to_keep, iscrowd=False)
        anns = self.coco_seg.loadAnns(ann_ids)
        
        # Initialize target dictionary
        target = {}
        boxes = []
        masks = []
        labels = []
        category_ids = []  # Keep original category IDs for reference
        
        # Process each annotation
        for ann in anns:
            if ann['area'] < self.min_area_threshold:
                continue
                
            # Get bounding box
            bbox = ann['bbox']  # [x, y, width, height] format
            # Convert to [x1, y1, x2, y2] format
            boxes.append([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])
            
            # Get mask
            mask = self.coco_seg.annToMask(ann)
            masks.append(torch.as_tensor(mask, dtype=torch.uint8))
            
            # Keep original category ID for reference
            category_ids.append(ann['category_id'])
            
            # For segmentation only, use class 1 for all foreground objects
            labels.append(1)  # 1 for foreground, 0 for background
        
        # Convert to tensor format
        if boxes:
            target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
            target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
            target["masks"] = torch.stack(masks)
            target["category_ids"] = torch.as_tensor(category_ids, dtype=torch.int64)  # original IDs for reference
        else:
            # Empty annotations
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros((0), dtype=torch.int64)
            target["masks"] = torch.zeros((0, image.shape[1], image.shape[2]), dtype=torch.uint8)
            target["category_ids"] = torch.zeros((0), dtype=torch.int64)
        
        target["image_id"] = torch.tensor([image_id])
        
        return image, target

In [7]:
dataset_train = CocoSegmentationDatasetMRCNN(
    config["train_image_dir"],
    config["train_annotation_file"],
    categories_to_keep=[84,31,52],
    min_area_threshold=655,
    output_dir="data/train"
)

dataset_val = CocoSegmentationDatasetMRCNN(
    config["val_image_dir"],
    config["val_annotation_file"],
    categories_to_keep=[84,31,52],
    min_area_threshold=655,
    output_dir="data/validation"
)

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: 'data/labels/labels_train.json'